## 本文主要记录多标签多分类模型的实现过程

### 整体流程
1. 依据数据格式，实现“数据读取”功能；（单元测试）
2. 基础主干网络ResNet-18实现；
3. 实现多标签多分类head，形成整体模型；（与2联合测试，绘制网络）
4. 多标签多分类模型损失函数实现；
5. 边边角角：配置与训练脚本、测试脚本、预测脚本，等等；（整体测试）
6. 进阶修改：损失函数修改，主干网络修改，等等。（整体测试）

### 构造训练数据集

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
input = np.random.normal(0, 1, [4, 2])
out_1 = np.random.normal(0, 1, [4, 1])
out_2 = np.random.normal(0, 1, [4, 1])
dataset = tf.data.Dataset.from_tensor_slices((input, (out_1, out_2)))
dataset = dataset.repeat().batch(2).prefetch(buffer_size=4)

# test
for i, data in enumerate(dataset):
    # (input, (out_1, out_2))
    print('=================================   {}  ======================================='.format(i))
    print('input is: \n', data[0])
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('output is: \n', data[1][0], '\n', data[1][1])
    if i >= 2:
        break

Instructions for updating:
Colocations handled automatically by placer.
=================================   0  =======================================
input is: 
 tf.Tensor(
[[-0.54730872  0.26720298]
 [-0.86050071  0.31083289]], shape=(2, 2), dtype=float64)
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
output is: 
 tf.Tensor(
[[1.89145406]
 [0.21500577]], shape=(2, 1), dtype=float64) 
 tf.Tensor(
[[-0.21285691]
 [ 0.6277284 ]], shape=(2, 1), dtype=float64)
=================================   1  =======================================
input is: 
 tf.Tensor(
[[ 1.00501827 -0.83485065]
 [ 1.67905237  1.30604547]], shape=(2, 2), dtype=float64)
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
output is: 
 tf.Tensor(
[[-1.10457522]
 [ 0.64685953]], shape=(2, 1), dtype=float64) 
 tf.Tensor(
[[-0.47960561]
 [-0.93504079]], shape=(2, 1), dtype=float64)
=================================   2  =======================================

### 建立keras模型
1. 定义骨干网络；
1. 实现多标签多分类head，形成整体模型；

In [2]:
from tensorflow import keras


def build_net(input_tensor):
    out1 = keras.layers.Dense(1, kernel_initializer='glorot_normal', activation='linear',
                              kernel_regularizer=keras.regularizers.l2(10))(input_tensor)
    out2 = keras.layers.Dense(1, kernel_initializer='glorot_normal', activation='linear',
                              kernel_regularizer=keras.regularizers.l2(10))(input_tensor)
    return [out1, out2]


feature_input = keras.layers.Input(shape=(2,), name='feature_input')
outputs = build_net(feature_input)
model = keras.models.Model(feature_input, outputs)

### 定义loss函数

In [3]:
import tensorflow as tf


def my_loss(y_dummy, pred):
    loss = tf.keras.losses.mean_absolute_error(y_dummy, pred)
    return loss


model.compile(loss=my_loss, optimizer='adam', loss_weights=[0.5, 0.5])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
feature_input (InputLayer)      (None, 2)            0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 1)            3           feature_input[0][0]              
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1)            3           feature_input[0][0]              
Total params: 6
Trainable params: 6
Non-trainable params: 0
__________________________________________________________________________________________________


### 训练与测试

In [4]:
# 训练
model.fit(dataset, epochs=5, steps_per_epoch=2, verbose=1)

# 测试
for i, data in enumerate(dataset):
    print('=================================   {}  ======================================='.format(i))
    print('input is: \n', data[0])
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('output is: \n', data[1][0], '\n', data[1][1])
    predictions = model.predict(np.array(data[0]))
    print('predictions is: \n', predictions[0], '\n', predictions[1])
    if i >= 2:
        break

Epoch 1/5
Instructions for updating:
Use tf.cast instead.
2/2 [==============================] - 0s 213ms/step - loss: 9.9042 - dense_loss: 1.0547 - dense_1_loss: 0.8552
Epoch 2/5
2/2 [==============================] - 0s 2ms/step - loss: 9.8423 - dense_loss: 1.0522 - dense_1_loss: 0.8527
Epoch 3/5
2/2 [==============================] - 0s 2ms/step - loss: 9.7809 - dense_loss: 1.0497 - dense_1_loss: 0.8506
Epoch 4/5
2/2 [==============================] - 0s 2ms/step - loss: 9.7199 - dense_loss: 1.0473 - dense_1_loss: 0.8484
Epoch 5/5
2/2 [==============================] - 0s 2ms/step - loss: 9.6592 - dense_loss: 1.0448 - dense_1_loss: 0.8463
=================================   0  =======================================
input is: 
 tf.Tensor(
[[-0.54730872  0.26720298]
 [-0.86050071  0.31083289]], shape=(2, 2), dtype=float64)
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
output is: 
 tf.Tensor(
[[1.89145406]
 [0.21500577]], shape=(2, 1), dtype=float64) 

更细致的debug（查看梯度、打印操作等），可看详细查看本工程。
